In [1]:
import pymysql
from collections import defaultdict
import pandas as pd
import numpy as np

In [15]:
#Database connection
conn = pymysql.connect(user='root', password='root', database='acb', host='localhost')

#Load the data into a Pandas DataFrame
query = "select * from acb.event join acb.game where acb.game.season = 2018 and journey <=22"
df = pd.read_sql(query, conn)
frames = []
team_possession_list = []
list_games = []
#GameDF dictionary
dict_of_games = {k: v for k, v in df.groupby('game_acbid')}
for game in dict_of_games.values():
    #For each team, we store a list with the possession events associated to that team. Besides, we only store the tuple (eventid, legend) associated to each event.
    #print("Game number: " + str(game.iloc[0]['game_acbid']))
    game_id = game.iloc[0]['game_acbid']
    cursor = conn.cursor() 
    sql = "SELECT id FROM game WHERE game_acbid ='%s'"
    cursor.execute(sql % game_id)
    game_id_bueno = cursor.fetchone()[0]
    list_games.append(game_id_bueno)
    list_games.append(game_id_bueno)
    game['eventid'] = list(range(1, game.shape[0]+1))
    possessions = defaultdict(list)
    #made2, made3 and turnover. These three categories are straightforward and we just insert them into each team:
    straightforward_elements = ["made2", "made3", "turnover"]
    straightforward_events = game[game["legend"].isin(straightforward_elements)]
    for _, event in straightforward_events.iterrows():
        possessions[event['team_id']].append((event['eventid'], event['legend']))
        
    #made1. We only include events of type made1 if they are the last free shot. That is, if extra_info is 1/1, 2/2 or 3/3:
    last_free_shot_elements = ["1/1", "2/2", "3/3"]
    last_free_shot_events = game[(game["legend"] == "made1") & (game["extra_info"].isin(last_free_shot_elements))]
    for _, event in last_free_shot_events.iterrows():
        possessions[event['team_id']].append((event['eventid'], event['legend']))
        
    #reb_def. Given a defensive rebound, the possession for the opponent team ends. Therefore, given a defensive rebound associated to team $i$, we insert such event to team $j$, where $i \not = j$.
    teams = list(possessions.keys())
    defensive_rebound_events = game[game["legend"] == "reb_def"]

    for _, event in defensive_rebound_events.iterrows():
        team = event['team_id']
        opponent_team = teams[(teams.index(team) + 1) % 2]
        possessions[opponent_team].append((event['eventid'], event['legend']))
    
    #technical fouls
    and_one_events = game[game["extra_info"] == "1/1"]

    for _, event in and_one_events.iterrows():
        if event['legend'] == 'made1': #1
            possessions[event['team_id']].remove((event['eventid'], event['legend']))
        else: # The shot was missed... What do I find first? Technical foul or made2/made3?
            for i in reversed(range(event['eventid'])): 
                if game[game.eventid == i].extra_info.item() == 'technical': #2
                    break
                elif (i, 'made2') in possessions[event['team_id']]: #3
                    possessions[event['team_id']].remove((i, 'made2'))
                    break
                elif (i, 'made3') in possessions[event['team_id']]: #3
                    possessions[event['team_id']].remove((i, 'made3'))
                    break
    
    #flagrant fouls
#    flagrant_elements = ["flagrant", "disqualifying"]
#    flagrant_events = game[game["extra_info"].isin(flagrant_elements)]
#    
#    for _, foul in flagrant_events.iterrows():
#        events = game[game["eventid"] > foul["eventid"]]
#        for _, event in events.iterrows():
#            if event["extra_info"] == "2/2" and event["legend"] == "miss1":
#                break
#            elif event["extra_info"] == "2/2" and event["legend"] == "made1":
#                print(possessions[event['team_code']])
#                possessions[event['team_code']].remove((event['eventid'], 'made1'))
#                break
#    
    for i, j in possessions.items():
        team_possession_list.append([i,len(j)])
        #print(i, len(j))
    
    
    def count_within_intervals(possessions, intervals):
        cont = 0
        idx_inter = 0
        idx_pos = 0
        left, right = intervals[idx_inter]
    
        while idx_pos < len(possessions):
            pos, _ = possessions[idx_pos]
            if pos < left: # smaller than the current interval, next pos...
                idx_pos+=1
            elif pos < right: # between current interval, ok!
                cont+=1
                idx_pos+=1
            else: # greater than current interval, new interval...
                idx_inter+=1
                if idx_inter == len(intervals): # no more intervals
                    break
                left, right = intervals[idx_inter]            
        return cont
    
    substitution_elements = ['sub_in', 'sub_out']
    substitutions_events = game[game['legend'].isin(substitution_elements)]
    
    
    individual_possessions = {}
    subs = substitutions_events.groupby("actor_id")
    for name, groups in subs:
        
        team = groups.iloc[0]['team_id']
        substitutions = list(groups['eventid'])
        if len(substitutions) % 2 == 1: # sometimes the last sub_out when the game ends is not registered...
            substitutions.append(game.eventid.max()+1)
        playing = list(zip(substitutions[::2], substitutions[1::2])) # intervals per player!
        individual_possessions[name] = count_within_intervals(possessions[team], playing)
        print(name)
        print(individual_possessions[name])
    
    
    s = pd.DataFrame(list(individual_possessions.items()), columns=['actor_id', 'possessions'])
    s['game_acbid'] = game_id
    s['actor_id'] = s['actor_id'].astype(int)
    frames.append(s)

df_possessions = pd.concat(frames)
#We want to extract the game_id
query = "select id, game_acbid from game where season >=2016"
df = pd.read_sql(query, conn)
joined_df = pd.merge(df_possessions, df, on='game_acbid', how='left')
result_df = joined_df.drop('game_acbid', 1)
result_df.rename(columns={'id':'game_id'}, inplace=True)
print(result_df.head())
df_possessions_team = pd.DataFrame(team_possession_list, columns=['team_id', 'possessions'])
df_possessions_team['game_id'] = list_games
print(df_possessions_team)

KeyboardInterrupt: 

In [16]:
#aggregate possessions by teams & players
df_possessions = df_possessions.groupby(by=['actor_id'], as_index=False)['possessions'].sum()
print(df_possessions.head(50))
df_possessions_team = df_possessions_team.groupby(by=['team_id'], as_index=False)['possessions'].sum()
print(df_possessions_team.head(50))

NameError: name 'df_possessions' is not defined

In [48]:
conn = pymysql.connect(user='root', password='root', database='acb', host='localhost')

query = "select game_id, team_id, possessions from participant join acb.game on acb.participant.game_id = acb.game.id where season = 2018 and journey <=22 and display_name like '%Equipo'"
df_possessions_team = pd.read_sql(query, conn)
df_possessions_team = df_possessions_team.groupby(by=['team_id'], as_index=False)['possessions'].sum()
print(df_possessions_team.head(50))

    team_id  possessions
0         1         1836
1         2         1770
2         3         1628
3         4         1583
4         5         1539
5         6         1671
6         7         1676
7         8         1597
8         9         1669
9        10         1724
10       11         1566
11       12         1581
12       13         1620
13       14         1546
14       15         1598
15       16         1573
16       17         1612
17       18         1543


In [49]:
conn = pymysql.connect(user='root', password='root', database='acb', host='localhost')

query = "select team_home_id, team_away_id, score_home, score_away, score_home_first, score_away_first, score_home_second, score_away_second, score_home_third, score_away_third, score_home_fourth, score_away_fourth from game where season = '2018' and journey <=22"
df_2018 = pd.read_sql(query, conn)

In [50]:
df_2018.shape

(205, 12)

In [51]:
df_1_off = df_2018[['team_home_id','score_home']]
df_2_off = df_2018[['team_away_id','score_away']]
df_1_off.rename(columns={'team_home_id':'team_id', 'score_home': 'puntos_favor'}, inplace=True)
df_2_off.rename(columns={'team_away_id':'team_id', 'score_away': 'puntos_favor'}, inplace=True)
df_score_off = pd.concat([df_1_off, df_2_off])

df_1_deff = df_2018[['team_home_id','score_away']]
df_2_deff = df_2018[['team_away_id','score_home']]
df_1_deff.rename(columns={'team_home_id':'team_id', 'score_away': 'puntos_contra'}, inplace=True)
df_2_deff.rename(columns={'team_away_id':'team_id', 'score_home': 'puntos_contra'}, inplace=True)
df_score_deff = pd.concat([df_1_deff, df_2_deff])

In [52]:
df_score_off = df_score_off.groupby(by=['team_id'], as_index=False)['puntos_favor'].sum()
print(df_score_off.shape)
df_score_deff = df_score_deff.groupby(by=['team_id'], as_index=False)['puntos_contra'].sum()
print(df_score_deff.shape)
df_scores = pd.merge(df_score_off, df_score_deff, on='team_id', how='left')

(18, 2)
(18, 2)


In [53]:
df_scores

,team_id,puntos_favor,puntos_contra
0,1,2227,1986
1,2,2187,1923
2,3,1981,1944
3,4,1832,1845
4,5,1718,1772
5,6,1969,1710
6,7,1885,1973
7,8,1731,1937
8,9,1983,1884
9,10,1951,1910


In [54]:
df_ratings = pd.merge(df_possessions_team, df_scores, on='team_id', how='left')
df_ratings.head(20)

,team_id,possessions,puntos_favor,puntos_contra
0,1,1836,2227,1986
1,2,1770,2187,1923
2,3,1628,1981,1944
3,4,1583,1832,1845
4,5,1539,1718,1772
5,6,1671,1969,1710
6,7,1676,1885,1973
7,8,1597,1731,1937
8,9,1669,1983,1884
9,10,1724,1951,1910


In [55]:
#Quiero nombres
query = "SELECT * FROM acb.team"
df_teamnames = pd.read_sql(query, conn)
df_teamnames.rename(columns={'id':'team_id'}, inplace=True)
df_ratings = pd.merge(df_ratings, df_teamnames, on='team_id', how='left')

In [56]:
df_ratings.head()

,team_id,possessions,puntos_favor,puntos_contra,team_acbid,founded_year
0,1,1836,2227,1986,MAD,1931
1,2,1770,2187,1923,BAR,1926
2,3,1628,1981,1944,RON,1977
3,4,1583,1832,1845,AND,1970
4,5,1539,1718,1772,OBR,1970


In [57]:
df_ratings['off_rating'] = (df_ratings['puntos_favor'] * 100) / df_ratings['possessions']
df_ratings['deff_rating'] = (df_ratings['puntos_contra'] * 100) / df_ratings['possessions']
df_ratings['net_rating'] = df_ratings['off_rating'] - df_ratings['deff_rating']
df_ratings = df_ratings.sort_values('net_rating', ascending = False).head(20)


In [58]:
df_ratings.head(20)

,team_id,possessions,puntos_favor,puntos_contra,team_acbid,founded_year,off_rating,deff_rating,net_rating
5,6,1671,1969,1710,BAS,1959,117.833633,102.333932,15.499701
1,2,1770,2187,1923,BAR,1926,123.559322,108.644068,14.915254
0,1,1836,2227,1986,MAD,1931,121.296296,108.169935,13.126362
8,9,1669,1983,1884,CAN,1939,118.813661,112.881965,5.931696
17,18,1543,1830,1783,PAM,1986,118.600130,115.554115,3.046014
11,12,1581,1810,1772,MAN,1931,114.484503,112.080961,2.403542
9,10,1724,1951,1910,JOV,1930,113.167053,110.788863,2.378190
2,3,1628,1981,1944,RON,1977,121.683047,119.410319,2.272727
3,4,1583,1832,1845,AND,1970,115.729627,116.550853,-0.821226
10,11,1566,1809,1846,BRG,1994,115.517241,117.879949,-2.362708


73.27272727272727